# Risk Parity: an Analytical Study

In [ ]:
! pip install fredapi

In [ ]:
import numpy as np
import pandas as pd
import requests
from fredapi import Fred
import yfinance as yf

In [ ]:
fred = Fred(api_key = 'ba8bf3a263d9f144e5f90ac707ecbd90')

# term premium is the 3mo/10Y diff
term_premium = fred.get_series(series_id = 'T10Y3M', observation_start='2000-01-01')
term_premium = pd.DataFrame(term_premium, columns=['TermPremium'])

# Get the 10Y T-Bill rate as the risk-free rate from FRED
risk_free_rate = fred.get_series('DGS10', observation_start='2000-01-01')
risk_free_rate = pd.DataFrame(risk_free_rate, columns = ['rate'])

In [ ]:
term_premium

,TermPremium
2000-01-03,1.10
2000-01-04,1.06
2000-01-05,1.18
2000-01-06,1.16
2000-01-07,1.14
...,...
2024-04-18,-0.82
2024-04-19,-0.83
2024-04-22,-0.80
2024-04-23,-0.84


In [ ]:
risk_free_rate

,rate
2000-01-03,6.58
2000-01-04,6.49
2000-01-05,6.62
2000-01-06,6.57
2000-01-07,6.52
...,...
2024-04-17,4.59
2024-04-18,4.64
2024-04-19,4.62
2024-04-22,4.62


In [ ]:
# Yahoo Finance Data Extraction

# Get the Russell 3000 historical data (using the ticker symbol ^RUA for Russell 3000)
russell_3000 = yf.download('IWV', start='2000-01-01', end='2024-01-01')['Adj Close']

# Get the S&P 500 historical data (using the ticker symbol ^GSPC for S&P 500)
sp500 = yf.download('SPY', start='2000-01-01', end='2024-01-01')['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
russell_3000 = pd.DataFrame(russell_3000, columns = ['Adj Close'])
sp500 = pd.DataFrame(sp500, columns = ['Adj Close'])

In [ ]:
# Convert annual rate to daily rate
risk_free_rate['Daily_Rate'] = risk_free_rate['rate'] / 100 / 252

# Calculate daily returns for Russell 3000 and S&P 500
russell_3000['Returns'] = russell_3000.pct_change()
sp500['Returns'] = sp500.pct_change()

In [ ]:
sp500.to_csv("SPY.csv")

In [ ]:
russell_3000

,Adj Close,Returns
Date,,
2000-05-26,50.444332,NaN
2000-05-30,51.889221,0.028643
2000-05-31,52.444099,0.010694
2000-06-01,52.590687,0.002795
2000-06-02,54.265850,0.031853
...,...,...
2023-12-22,272.272552,0.002534
2023-12-26,273.549316,0.004689
2023-12-27,274.177734,0.002297


In [ ]:
# Align the dataframes by date
aligned_data = pd.concat([
    russell_3000.rename(columns={'Returns': 'Russell_Returns'}),
    sp500.rename(columns={'Returns': 'SP500_Returns'}),
    risk_free_rate[['Daily_Rate']]
], axis=1, join='inner')

In [ ]:
# Calculate excess returns
aligned_data['Russell_Excess'] = aligned_data['Russell_Returns'] - aligned_data['Daily_Rate']
aligned_data['SP500_Excess'] = aligned_data['SP500_Returns'] - aligned_data['Daily_Rate']

# Calculate Sharpe Ratios
sharpe_russell = aligned_data['Russell_Excess'].mean() / aligned_data['Russell_Returns'].std() * np.sqrt(252)
sharpe_sp500 = aligned_data['SP500_Excess'].mean() / aligned_data['SP500_Returns'].std() * np.sqrt(252)

print(f"Sharpe Ratio for Russell 3000: {sharpe_russell}")
print(f"Sharpe Ratio for S&P 500: {sharpe_sp500}")

Sharpe Ratio for Russell 3000: 0.2814210966405921
Sharpe Ratio for S&P 500: 0.27438798316619406


In [ ]:
aligned_data

,Adj Close,Russell_Returns,Adj Close,SP500_Returns,Daily_Rate,Russell_Excess,SP500_Excess
2000-05-26,50.444332,NaN,89.030701,0.001133,0.000251,NaN,0.000882
2000-05-30,51.889221,0.028643,91.933876,0.032609,0.000253,0.028390,0.032356
2000-05-31,52.444099,0.010694,92.135452,0.002193,0.000250,0.010444,0.001943
2000-06-01,52.590687,0.002795,93.748329,0.017505,0.000246,0.002549,0.017259
2000-06-02,54.265850,0.031853,95.381401,0.017420,0.000244,0.031609,0.017176
...,...,...,...,...,...,...,...
2023-12-22,272.272552,0.002534,472.182892,0.002010,0.000155,0.002379,0.001855
2023-12-26,273.549316,0.004689,474.176697,0.004223,0.000154,0.004535,0.004068
2023-12-27,274.177734,0.002297,475.034058,0.001808,0.000150,0.002147,0.001658
2023-12-28,274.117889,-0.000218,475.213501,0.000378,0.000152,-0.000371,0.000225


In [ ]:
russell_3000.to_csv('Russell.csv')

In [ ]:
risk_free_rate.to_csv("rf.csv")

In [ ]:
term_premium.to_csv("tp.csv")